In [257]:
import MySQLdb
import MySQLdb.cursors
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as LinR
import numpy as np
import sys
import re
%matplotlib inline
import matplotlib as plt
from __future__ import division
from sklearn.decomposition import PCA

In [258]:
conn = MySQLdb.connect(
            user="root",
            passwd="",
            db="Player_Team_Data",
            cursorclass=MySQLdb.cursors.DictCursor)

In [259]:
df_all_players = pd.read_csv('~/Insight/Players100.csv')
    # Remove rows that were separated by random 'Player' entries
df_all_players = df_all_players[df_all_players.Name != 'Player']

name_list=list(df_all_players.values)
new_name_list = []

for name in name_list:
        # convert entries to strings
    name = str(name)
    new_name_list.append(name)

unique_name_list=list(set(new_name_list))
unique_name_list.sort()
name_list_final = []
for name in unique_name_list:
    match = re.search('([\w\.\-\']+) ([\w\.\-]+)', name)
    prename = match.group(1).lower()[1:] + ' ' + match.group(2).lower()
    name_list_final.append(prename)

name_list_fin = [x for x in name_list_final if x != 'george hill']

validation={}

 

def get_fanduel(predict,avg,tru):
    fanduel_pre = predict[0]+predict[2]*1.2+predict[3]*1.5+predict[4]*2+predict[5]*2-predict[6]
    fanduel_avg = avg[0]+avg[2]*1.2+avg[3]*1.5+avg[4]*2+avg[5]*2-avg[6]
    fanduel_tru = tru[0]+tru[2]*1.2+tru[3]*1.5+tru[4]*2+tru[5]*2-tru[6]
    return fanduel_pre, fanduel_avg, fanduel_tru

def get_trend(fan_pre,fan_avg,fan_tru):
    
    if (fan_pre > fan_avg) and (fan_tru > fan_avg):
        val = 1
    elif (fan_pre < fan_avg) and (fan_tru < fan_avg):
        val = 1
    elif (fan_pre < fan_avg) and (fan_tru > fan_avg):
        val = 0
    elif (fan_pre > fan_avg) and (fan_tru < fan_avg):    
        val = 0
    else:
        val = 2
    return val

In [260]:
Acc_dict ={}
for player in name_list_fin:
    print player
    cmd_Rk= 'SELECT Rk FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\')  AND Year IN (\'2015\');'
    df_Rk = pd.read_sql(cmd_Rk, con=conn) 
    Accuracy_PD=pd.DataFrame(columns=['PTS','3P','TRB','AST','STL','BLK','TOV'])
    Truth_PD=pd.DataFrame(columns=['PTS','3P','TRB','AST','STL','BLK','TOV'])
    for Rki in df_Rk.Rk:
        Rk = str(Rki)
        cmd_target_2015 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+' ;'
        cmd_target_2014 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+' ;'
        cmd_train_2015 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+';'
        cmd_train_2014 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+';'
        cmd_operate = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+';'
        cmd_truth = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+' ;'
        cmd_min_2015 = 'SELECT MP FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+';'   
        cmd_min_2014 = 'SELECT MP FROM NBA_player_data_MP WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk >= '+Rk+';'   


        df_target_2015 = pd.read_sql(cmd_target_2015, con=conn) 
        df_target_2014 = pd.read_sql(cmd_target_2014, con=conn)
        df_min_2015 = pd.read_sql(cmd_min_2015, con=conn) 
        df_min_2014 = pd.read_sql(cmd_min_2014, con=conn) 
        df_train_2015 = pd.read_sql(cmd_train_2015, con=conn) 
        df_train_2014 = pd.read_sql(cmd_train_2014, con=conn) 
        df_operate = pd.read_sql(cmd_operate, con=conn) 
        df_truth = pd.read_sql(cmd_truth, con=conn) 
        df_truth = df_truth.applymap(lambda x: float(x))
       

        df_min = pd.concat([df_min_2014, df_min_2015],ignore_index=True)
        df_target=pd.concat([df_target_2014, df_target_2015],ignore_index=True)
        df_train=pd.concat([df_train_2014, df_train_2015],ignore_index=True)
        df_target = df_target.applymap(lambda x:float(x))
        df_train = df_train.applymap(lambda x:float(x))
        df_target_2015 = df_target_2015.applymap(lambda x: float(x))

        df_inquire = df_operate.applymap(lambda x:float(x))
        df_train_plus_inquire=pd.concat([df_train, df_inquire])
        if int(Rk) < 20:
            average_stats=df_target.mean().apply(lambda x: round(x)) 
        elif int(Rk) >= 20:
            average_stats=df_target_2015.mean().apply(lambda x: round(x))
        df_bool = df_target.copy()
        df_bool.PTS = df_target.PTS.apply(lambda x: x<average_stats.PTS.mean())
        df_bool['3P'] = df_target['3P'].apply(lambda x: x<average_stats['3P'].mean())
        df_bool.TRB = df_target.TRB.apply(lambda x: x<average_stats.TRB.mean())
        df_bool.AST = df_target.AST.apply(lambda x: x<average_stats.AST.mean())
        df_bool.STL = df_target.STL.apply(lambda x: x<average_stats.STL.mean())
        df_bool.BLK = df_target.BLK.apply(lambda x: x<average_stats.BLK.mean())
        df_bool.TOV = df_target.TOV.apply(lambda x: x<average_stats.TOV.mean())
        df_truth_bool = df_truth.copy()

        df_truth_bool.PTS = df_truth.PTS.apply(lambda x: x<average_stats.PTS.mean())
        df_truth_bool['3P'] = df_truth['3P'].apply(lambda x: x<average_stats['3P'].mean())
        df_truth_bool.TRB = df_truth.TRB.apply(lambda x: x<average_stats.TRB.mean())
        df_truth_bool.AST = df_truth.AST.apply(lambda x: x<average_stats.AST.mean())
        df_truth_bool.STL = df_truth.STL.apply(lambda x: x<average_stats.STL.mean())
        df_truth_bool.BLK = df_truth.BLK.apply(lambda x: x<average_stats.BLK.mean())
        df_truth_bool.TOV = df_truth.TOV.apply(lambda x: x<average_stats.TOV.mean())

        df_raw = df_train_plus_inquire.reindex()
        df_raw_scaled = df_raw.copy()
        df_raw_scaled = df_raw_scaled.applymap(lambda x: np.log(x))
        df_raw_transform = df_raw.copy()

        df_raw_scaled = df_raw_scaled.apply(lambda x:preprocessing.StandardScaler().fit(x).transform(x))
        df_raw_transform = df_raw_transform.apply(lambda x:preprocessing.StandardScaler().fit(x))

        #df_train_reduced=pd.DataFrame(train_reduced)
        df_evaluate = df_raw_scaled.tail(1)
        df_train_scaled = df_raw_scaled.iloc[:-1]

        rf = RandomForestClassifier(n_estimators=100)
        rf.fit(df_train_scaled, df_bool)
        predictions = pd.DataFrame(rf.predict(df_evaluate),columns=['PTS','3P','TRB','AST','STL','BLK','TOV']).applymap(lambda x:bool(x))
        pd_accuracy = predictions == df_truth_bool
        Accuracy_PD = pd.concat([Accuracy_PD, pd_accuracy],ignore_index=True)
        Truth_PD = pd.concat([Truth_PD, pd_accuracy],ignore_index=True)
    Ast_a= Accuracy_PD['AST'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Reb_a= Accuracy_PD['TRB'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Tp_a= Accuracy_PD['3P'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Pts_a= Accuracy_PD['PTS'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Stl_a= Accuracy_PD['STL'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Blk_a= Accuracy_PD['BLK'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Tov_a= Accuracy_PD['TOV'].value_counts(sort=False)[1]/len(Accuracy_PD)
    Acc_list = [Pts_a, Tp_a, Reb_a, Ast_a, Stl_a, Blk_a, Tov_a]
    Acc_dict[player]=Acc_list

al horford
al jefferson
amir johnson
andre drummond
anthony davis
arron afflalo
avery bradley
ben mclemore
blake griffin
bradley beal
brandon bass
brandon knight
brook lopez
carlos boozer
carmelo anthony
chris bosh
chris paul
corey brewer
courtney lee
damian lillard
danny green
david west
deandre jordan
demarcus cousins
demarre carroll
demar derozan
deron williams
derrick favors
dion waiters
dirk nowitzki
draymond green
dwyane wade
enes kanter
eric bledsoe
gerald green
gerald henderson
giannis antetokounmpo
goran dragic
gordon hayward
gorgui dieng
harrison barnes
isaiah thomas
jamal crawford
james harden
jarrett jack
jeff green
jeff teague
jeremy lin
jimmy butler
joe johnson
john wall
jonas valanciunas
jordan hill
josh smith
kawhi leonard
kenneth faried
kentavious caldwell-pope
kevin durant
kevin love
khris middleton
klay thompson
kyle korver
kyle lowry
kyrie irving
lamarcus aldridge
lebron james
louis williams
luol deng
manu ginobili
marc gasol
marcin gortat
mario chalmers
markieff mo

In [261]:
df_Acc = pd.DataFrame(Acc_dict,index=['PTS','3P','TRB','AST','STL','BLK','TOV'])

In [262]:
df_AccT=df_Acc.T

In [274]:
df_AccT.TOV.mean()

0.59411991304273981

In [264]:
df_AccT.AST.mean()

0.56279207045811375

In [265]:
score =[]
for i,f in enumerate(f_pre):
    s=get_trend(f,f_avg[i],f_tru[i])
    score.append(s)


In [266]:
x = [1,2,3]
plt.pyplot.bar( x,[predictions[0],average_stats[0] , true_stats[0]])

KeyError: 0

In [ ]:
a = [x for x in score if x != 2]

In [ ]:
a.count(1)/len(a)



In [ ]:
def get_min(x):
    match = re.search('(\w+):(\w+)', x)
    minute = float(match.group(1))
    second = float(match.group(2))/60
    timet = minute+second
    return timet

In [ ]:
df_minx = df_min.applymap(lambda x: get_min(x))
df_minx

In [ ]:
df_minx.std()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

In [ ]:
plt.rcParams['figure.figsize'] = 12, 8 
fig, ax = plt.subplots()
ind = np.arange(len(f_pre[30:62])) 
width = 0.35
rects1 = ax.bar(ind, f_pre[30:62], width, color='r',label='Prediction')

rects2 = ax.bar(ind+width, f_tru[30:62], width, color='k',label ='Truth')

rects3 = ax.plot(f_avg[30:62],color='y',linewidth=10,label='Average')

ax.legend(loc='upper right',prop={'size':20})
plt.xlabel('Game of the Season',fontsize=30)
plt.ylabel('Fanduel Metric',fontsize=30)
plt.title('Gordon Hayward',fontsize=35) 
plt.xticks('xtick', labelsize=30) 
plt.yticks('ytick', labelsize=30) 
plt.ylim((0,60))
plt.savefig('GH.png')